In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [2]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.regularizers import l1,l2,l1_l2

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [4]:
df_train = pd.read_csv('df_train.csv')
df_train

,5160399000.0,5160532000.0,5160665000.0,5160798000.0,5160931000.0,5161064000.0,5161197000.0,5161330000.0,5161463000.0,5161596000.0,...,5178354000.0,5178487000.0,5178620000.0,5178753000.0,5178886000.0,5179019000.0,5179152000.0,5179285000.0,5179418000.0,target
0,7.172340,13.318998,15.115269,12.120115,12.898385,13.807718,15.543046,14.583542,11.757287,12.031413,...,12.937673,12.112266,15.071740,15.653405,15.110099,14.973473,13.874439,15.763504,14.146375,0
1,8.772359,11.914471,14.203160,11.378137,11.834629,12.236748,14.267263,14.501087,11.912227,11.376563,...,11.545565,10.424321,13.245390,13.355729,12.802243,13.800442,12.812164,14.272315,12.451538,0
2,8.391178,11.156736,14.503078,12.438627,12.427057,13.169839,14.929343,14.835480,12.524612,12.274352,...,11.538292,10.504514,12.372451,12.029099,12.614220,13.682375,11.623414,13.791919,12.385667,0
3,6.163323,10.432897,15.272127,13.891915,12.910382,9.931044,12.755279,12.925886,12.226484,13.552130,...,11.433129,11.008790,13.047578,12.179321,12.295100,14.118652,12.313926,12.769601,11.562403,0
4,8.363369,11.315283,14.668475,13.853479,13.341522,10.920540,13.074407,12.922540,11.044026,13.024234,...,11.894255,11.838393,13.996025,13.175538,12.259200,13.906978,11.991285,12.603871,11.924631,0
5,8.240896,13.739551,16.909309,15.751151,14.918041,13.021907,15.742979,13.975338,11.833842,15.607986,...,12.337350,14.301595,16.914551,16.342760,15.981591,16.695617,13.586391,15.972772,15.946127,0
6,8.199476,14.391475,16.913791,14.984069,15.889930,14.243103,16.211843,14.473472,11.977827,15.824573,...,12.634924,14.355559,16.362179,16.782259,16.037976,16.848422,14.666279,16.549522,14.835597,0
7,8.853883,14.978434,16.108675,14.508603,16.282452,15.486989,16.696201,15.084880,12.343903,16.218308,...,14.137195,14.221792,16.397888,17.018057,15.751225,16.043181,13.503233,17.207232,14.548845,0
8,11.864605,14.443004,15.056640,13.854646,16.628577,15.871353,16.604690,14.021480,11.470074,14.144264,...,14.168295,14.729098,16.445488,17.023000,16.745621,16.771568,13.254674,16.521037,14.461398,0
9,12.044046,13.901843,14.996692,13.340502,15.901869,16.480884,16.409730,13.628276,11.467872,13.878743,...,13.211286,14.162416,16.526667,17.388181,16.718023,16.690297,13.601106,16.481902,13.965977,0


In [5]:
df_med = df_train.drop(df_train.iloc[:, :36], axis = 1) 
df_med.drop(df_med.iloc[:, 72:108], inplace = True, axis = 1) 

df_small = df_train.drop(df_train.iloc[:, :54], axis = 1) 
df_small.drop(df_small.iloc[:, 36:90], inplace = True, axis = 1) 

df_smaller = df_train.drop(df_train.iloc[:, :64], axis = 1) 
df_smaller.drop(df_smaller.iloc[:, 18:80], inplace = True, axis = 1) 

# df_med.to_csv(r'df_med_train.csv', index=False)
# df_small.to_csv(r'df_small_train.csv', index=False)
# df_smaller.to_csv(r'df_smaller_train.csv', index=False)

In [6]:
df_smooth = df_train.T
df_med = df_med.T
df_small = df_small.T
df_smaller = df_smaller.T

In [7]:
sequences_smooth = list()
for i in range(df_smooth.shape[1]):
    values = df_smooth.iloc[:-1,i].values
    sequences_smooth.append(values)
targets_smooth = df_smooth.iloc[-1, :].values

sequences_med = list()
for i in range(df_med.shape[1]):
    values = df_med.iloc[:-1,i].values
    sequences_med.append(values)
targets_med = df_med.iloc[-1, :].values

sequences_small = list()
for i in range(df_small.shape[1]):
    values = df_small.iloc[:-1,i].values
    sequences_small.append(values)
targets_small = df_small.iloc[-1, :].values

sequences_smaller = list()
for i in range(df_smaller.shape[1]):
    values = df_smaller.iloc[:-1,i].values
    sequences_smaller.append(values)
targets_smaller = df_smaller.iloc[-1, :].values

In [8]:
targets = targets_smooth

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(targets)
encoded_y = encoder.transform(targets)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [10]:
targets = dummy_y

In [11]:
X_train_smooth, X_train_med, X_train_small, X_train_smaller, y_train = sequences_smooth, sequences_med, sequences_small, sequences_smaller, targets

In [12]:
from sklearn.preprocessing import StandardScaler
sc1 = StandardScaler()
X_train_smooth = sc1.fit_transform(X_train_smooth)

sc2 = StandardScaler()
X_train_med = sc2.fit_transform(X_train_med)

sc3 = StandardScaler()
X_train_small = sc3.fit_transform(X_train_small)

sc4 = StandardScaler()
X_train_smaller = sc4.fit_transform(X_train_smaller)

from sklearn.externals.joblib import dump, load
dump(sc1, 'std_scaler_smooth.bin', compress=True)
dump(sc2, 'std_scaler_med.bin', compress=True)
dump(sc3, 'std_scaler_small.bin', compress=True)
dump(sc4, 'std_scaler_smaller.bin', compress=True)

/home/hongyu/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['std_scaler_smaller.bin']